## In this lecture:
1. Download the data
2. Preprocess the data

    2.1. Scalling
    
    2.2. Spliting
    
    2.3. Batching

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Downloading  and preprocessing data

In [2]:
BUFFER_SIZE = 70_000
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [3]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info=True, as_supervised = True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    
    return image, label

In [6]:
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [7]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [9]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [10]:
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [11]:
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples) 

## Creating Model and Training it

1. Define the architecture of the model
2. compile the model
3. train the model

(conv, 5x5, 50, ReLu) => (MaxPool, 2x2, Stride2) => (conv, 3x3, 50, ReLu) => (MaxPool, 2x2, Stride2) => Flatten => (Dense, 10, Softmax)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50, 5, activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)    
])

In [13]:
model.summary(line_length=75)

Model: "sequential"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d (Conv2D)                 (None, 24, 24, 50)            1300        
                                                                           
 max_pooling2d (MaxPooling2D)    (None, 12, 12, 50)            0           
                                                                           
 conv2d_1 (Conv2D)               (None, 10, 10, 50)            22550       
                                                                           
 max_pooling2d_1 (MaxPooling2D)  (None, 5, 5, 50)              0           
                                                                           
 flatten (Flatten)               (None, 1250)                  0           
                                                                           
 dense (Dense)                   (None, 10)                    12510

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

In [15]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping( 
    monitor = 'val_loss',
    mode= 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights=True
)

In [ ]:
model.fit(
    train_data,
    epochs = NUM_EPOCHS,
    callbacks = [early_stopping],
    verbose=2
)

Epoch 1/20
